# EKDQ - Evolution of KDQ Trees

This repository holds some of my recent experiments in relation to an interesting data structure - the *kdq*-tree.
Specifically, I am interested in it's applications for change detection, building on earlier work by Dasu et al. [[1]](http://www.cse.ust.hk/~yike/datadiff/datadiff.pdf)

## Change detection in multidimensional data

This is what we want to achieve. Given a very high dimensional data set (or stream of data), without any labels, can we
tell if the nature of the data changes significantly? The more dimensions you add, the tricker this becomes. 

If you're interested in this problem, you probably need professional help. But beforehand, you may be interested in my PhD thesis [[2]](https://www.researchgate.net/profile/Will_Faithfull/publication/328580385_Unsupervised_Change_Detection_in_Multivariate_Streaming_Data/links/5bd70ba74585150b2b8e6b2f/Unsupervised-Change-Detection-in-Multivariate-Streaming-Data.pdf).

## KD"Q" Trees?

Let's say we're interested in modelling the *relative* distributions of two sets of data, $W1, W2$, which we can make few assumptions about.
If you separate the data into equal sized bins, you could generate a histogram with an empirical distribution:

![](https://upload.wikimedia.org/wikipedia/commons/1/1d/Example_histogram.png)

How do you do this for more than one dimension? One way is to construct a QuadTree:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Quad_tree_bitmap.svg/380px-Quad_tree_bitmap.svg.png)

Ok, that's great for two dimensions, but what about 3? You need an octree, with 8 children per node. What about 50? 1000? You get the picture.

The traditional approach for extending spatial partitioning trees is KD-trees. They partition a *k*-dimensional space by iterating through the *k* axes as they
make splits from the points. In 2D, they might look like this:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Kdtree_2d.svg/370px-Kdtree_2d.svg.png)

These are great, and scale well. However, they have an undesirable quality for our histogramming idea. They produce very inconsistently sized cells.

A *kdq* tree takes this idea of axis iteration, and instead simply splits at the midpoint of an existing cell. In 2D, they might look like this: 

![](https://pbs.twimg.com/media/EjBjtn4XYAsfY8m?format=png&name=small)

## EKDQ

Dasu et al.[[1]](http://www.cse.ust.hk/~yike/datadiff/datadiff.pdf) produce a tree from $W1$, use this to make a histogram, and then compare this to a histogram produced from $W2$, using the Kullback-Leibler divergence between the two. For empirical distributions (histograms) $P$ and $Q$:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/4958785faae58310ca5ab69de1310e3aafd12b32)

Whilst being the obvious approach, this does have weaknesses.

- No natural threshold for the KL Divergence. The authors suggest bootstrapping a threshold with monte carlo methods.
- KL Divergence treats changes between the cells as equally important. A change in count in a small cell is probably more significant than a change in a large cell.

As an alternative, I propose here a novel change detection scheme based on monitoring the *evolution* of the KDQ tree over time.

Consider this as a streaming problem. First, we take a sample of *m* datums from the stream, and use this to build a KDQ tree. Now we sample another *m* datums as as $W1$. If we fix the cardinality of the KDQ tree to *m*, deleting the oldest datum as we insert another, then each operation has the following possible outcomes:

- No change in tree structure
- One or more cells split
- Two cells merges

Playing $W1$ into the tree generates a set of events $E_{W1}$, where $|E_{W1}| = |W1| = m$.

```
U = unchanged
S = split
M = merge

M,U,U,U,S,S,U,...,U,U,S,S,S,U,U,U
```

If we now sample another *m* events as $W2$, and play these into the tree, we get another set of events $E_{W2}$, $|E_{W2}| = |E_{W1}| = |W1| = |W2| = m$.

The hypothesis is that if $W1$ and $W2$ are drawn from the same distribution, then $E_{W2}$ and $E_{W1}$ should be similar, as data from an unchanged distribution should cause minimal structural changes in the tree.

We can model these events with a multinomial distribution of parameters $p$ and $k$.

\begin{align}
\frac{n!}{x_1!, ..., x_k!} p_{1}^{x_1},...,p_{k}^{x_k}
\end{align}

Taking the empirical $p$ and $k$ from $E_{W1}$, we can assess the log-likelihood of the data from $E_{W2}$ with respect to these parameters.

The multinomial distribution offers a simplification of the log-likelihood ratio, the G-test.

With $m$ as the sample size, $E_i$ as the expected count of event type $i$, and $O_i$ as the observed count of event type $i$, the G statistic is given by

\begin{align}
G = -2\sum^{m}_{i=1}O_i\ln\left(\frac{E_i}{O_i}\right)
\end{align}

According to Wilk's theorem, as $m \to \infty$, $G$ asymptotically approaches a $\chi^2$ distribution under the null hypothesis. This provides us with a ready-made threshold with which to reject the null hypothesis.

1. [Dasu, Tamraparni, et al. "An information-theoretic approach to detecting changes in multi-dimensional data streams." In Proc. Symp. on the Interface of Statistics, Computing Science, and Applications. 2006.](http://www.cse.ust.hk/~yike/datadiff/datadiff.pdf)
2. [Faithfull, William. Unsupervised Change Detection in Multivariate Streaming Data. Diss. Bangor University, 2018.](https://www.researchgate.net/profile/Will_Faithfull/publication/328580385_Unsupervised_Change_Detection_in_Multivariate_Streaming_Data/links/5bd70ba74585150b2b8e6b2f/Unsupervised-Change-Detection-in-Multivariate-Streaming-Data.pdf)